In [266]:
import tensorflow as tf
import numpy as np
from common import *

import logging
import os, time
import tflearn
import tflearn.helpers.summarizer as s
from io import StringIO
import copy
import pickle
from functools import partial
import datetime

from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.DEBUG)

# Prepare word vectors

In [3]:
w2v_model = Word2Vec.load(join(DATA_FOLDER, 'vectors/w2v_model_300_w10'))

2017-08-23 10:20:03,354 : INFO : loading Word2Vec object from ../data/vectors/w2v_model_300_w10
2017-08-23 10:20:06,546 : INFO : loading wv recursively from ../data/vectors/w2v_model_300_w10.wv.* with mmap=None
2017-08-23 10:20:06,548 : INFO : loading syn0 from ../data/vectors/w2v_model_300_w10.wv.syn0.npy with mmap=None
2017-08-23 10:20:06,849 : INFO : setting ignored attribute syn0norm to None
2017-08-23 10:20:06,851 : INFO : loading syn1neg from ../data/vectors/w2v_model_300_w10.syn1neg.npy with mmap=None
2017-08-23 10:20:07,159 : INFO : setting ignored attribute cum_table to None
2017-08-23 10:20:07,160 : INFO : loaded ../data/vectors/w2v_model_300_w10


In [4]:
word_embeddings = w2v_model.wv.syn0.copy()
index2word = copy.deepcopy(w2v_model.wv.index2word)
del w2v_model

In [5]:
index2word.insert(0, 'PAD')
with open(join(DATA_FOLDER, "dictionary.pickle"), "wb") as output_file:
    pickle.dump(index2word, output_file)

In [6]:
stds = np.apply_along_axis(np.std, 1, word_embeddings)
pd.Series(stds).describe()

count    680760.000000
mean          0.193705
std           0.070574
min           0.000910
25%           0.137368
50%           0.178099
75%           0.244486
max           0.744395
dtype: float64

In [7]:
# 0.34 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
pad_vec = np.random.uniform(-0.34,0.34, word_embeddings.shape[1])
np.std(pad_vec)

0.18944918087888768

In [8]:
word_embeddings = np.insert(word_embeddings, 0, pad_vec, axis=0)

In [9]:
np.save(join(DATA_FOLDER, 'word_embeddings_%s.npy' % word_embeddings.shape[1]), word_embeddings)

# Load input data

In [267]:
word_embeddings = np.load(join(DATA_FOLDER, 'word_embeddings_300.npy'))
with open(join(DATA_FOLDER, "dictionary.pickle"), "rb") as input_file:
    index2word = pickle.load(input_file)

In [268]:
ids = glob('../data/corpus/**.txt')
with open(join(DATA_FOLDER, 'sims.json'), 'r') as f:
    sims = json.load(f)

In [269]:
# random select nagative examples
import random

def full_name(_id):
    return join(DATA_FOLDER, 'corpus/%s.txt' % _id)

def random_triples(sims, ids, num_epochs=1, seed=0):
    """
    Get random triples, select negatives at random in each epoch.
    Output: [anchor, positive, negative]
    """
    random.seed(0)
    ixs = list(range(len(ids)))
    for ep in range(num_epochs):
        random.shuffle(ixs)
        it = iter(ixs)
        for k, v in tqdm(sims.items()):
            exclude = [full_name(i) for i in [k] + v]
            for vi in v:
                ix = next(it)
                _neg = ids[ix]
                while _neg in exclude:
                    ix = next(it)
                    _neg = ids[ix]
                yield [full_name(k), full_name(vi), _neg]

# Input pipline

In [270]:
def parse_csv(text):
    with tf.name_scope('parse_csv'):
        strings = tf.string_split([text], delimiter='\n')
        raw_nums = tf.string_split(strings.values)
        nums = tf.string_to_number(raw_nums.values, tf.int32)
        dense = tf.sparse_to_dense(
            raw_nums.indices, raw_nums.dense_shape, nums, default_value=0)
        dense.set_shape(raw_nums.get_shape())
    return dense

def read_input_tuple(filename_queue):
    with tf.name_scope('read_input_tuple'):
        fnames = filename_queue.dequeue()
        example = []
        for fn in tf.unstack(fnames):
            record_string = tf.read_file(fn)
            arr = parse_csv(record_string)
            example.append(arr)
        example.append(fnames)
    return example

def input_pipeline(triples, batch_size, num_epochs=1, num_threads=cpu_count, shuffle=True):
    filename_queue = tf.train.input_producer(
        triples, num_epochs=num_epochs, capacity=32, shuffle=shuffle, seed=0)
    example = read_input_tuple(filename_queue)

    min_after_dequeue = 10000
    capacity = min_after_dequeue + 3 * batch_size
    anchor, positive, negative, fnames = tf.train.batch(
        example,
        batch_size=batch_size,
        capacity=capacity,
        dynamic_pad=True,
        #         allow_smaller_final_batch=True,
        num_threads=num_threads)
    return anchor, positive, negative, fnames

# Model definition

In [290]:
class TextCNN(object):
    def __init__(self,
                 n_sents,
                 n_words,
                 vocab_size,
                 embedding_size,
                 sent_filter_sizes=[2, 3, 4, 5],
                 sent_nb_filter=15,
                 doc_filter_sizes=[1, 2, 3],
                 doc_nb_filter=10,
                 sent_kmax=10,
                 doc_kmax=10,
                 learning_rate=0.001):
        with tf.name_scope('init_model'):
            self.n_sents = n_sents
            self.n_words = n_words
            self.vocab_size = vocab_size
            self.embedding_size = embedding_size
            self.sent_filter_sizes = sent_filter_sizes
            self.sent_nb_filter = sent_nb_filter
            self.doc_filter_sizes = doc_filter_sizes
            self.doc_nb_filter = doc_nb_filter
            self.sent_kmax = sent_kmax
            self.doc_kmax = doc_kmax
            self.learning_rate = learning_rate
            self.sess = tf.get_default_session()
            
            assert (self.sess is not None and not self.sess._closed)
            
            self.global_step = tf.get_variable("global_step", initializer=tf.constant(0), trainable=False)
            self.optimizer = tf.train.AdamOptimizer(
                learning_rate=learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-08)

            # Embedding layer
            with tf.device('/cpu:0'), tf.name_scope("embedding"):
                self.LT = tf.get_variable('LT',
                    initializer=tf.constant(0.0, shape=[vocab_size, embedding_size]),
                    trainable=False)

                self.embedding_placeholder = tf.placeholder(
                    tf.float32, [self.vocab_size, self.embedding_size])
                self.embedding_init = self.LT.assign(self.embedding_placeholder)

            with tf.variable_scope('sent'):
                self._create_sharable_weights(sent_filter_sizes, embedding_size,
                                              sent_nb_filter)
                self.sent_embedding_size = tf.convert_to_tensor(
                    sent_kmax * sent_nb_filter * len(sent_filter_sizes))

            with tf.variable_scope('doc'):
                self._create_sharable_weights(doc_filter_sizes,
                                              self.sent_embedding_size.eval(),
                                              doc_nb_filter)
                self.doc_embedding_size = tf.convert_to_tensor(
                    doc_kmax * doc_nb_filter * len(doc_filter_sizes))

        print('sent_embedding_size %s' % self.sent_embedding_size.eval())
        print('doc_embedding_size %s' % self.doc_embedding_size.eval())

            
    def inference(self, X):
        """ This is the forward calculation from batch X to doc embeddins """
        
        embedded_words = tf.nn.embedding_lookup(self.LT, X)
        embedded_words_expanded = tf.expand_dims(embedded_words, -1)
        
        with tf.variable_scope('sent'):

            def convolv_on_sents(embeds):
                return self._convolv_on_embeddings(
                    embeds, self.sent_filter_sizes, self.sent_nb_filter,
                    self.sent_kmax)

            # iter over each document
            self.sent_embed = tf.map_fn(
                convolv_on_sents,
                embedded_words_expanded,
                parallel_iterations=10,
                name='iter_over_docs')
            # sent_embed shape is [batch, n_sent, sent_sent_kmax*sent_nb_filter*len(sent_filter_sizes), 1]

        with tf.variable_scope('doc'):
            # finally, convolv on documents
            self.doc_embed = self._convolv_on_embeddings(
                self.sent_embed, self.doc_filter_sizes, self.doc_nb_filter,
                self.doc_kmax)
            # doc_embed shape is [batch, doc_kmax*doc_nb_filter*len(doc_filter_sizes), 1]

        doc_embed_normalized = tf.nn.l2_normalize(
            self.doc_embed, dim=1, name='doc_embed_normalized')

        return doc_embed_normalized

    def loss(self, X):
        with tf.name_scope("loss"):
            doc_embed_normalized = self.inference(X)
            self.anchor, self.positive, self.negative = tf.unstack(
                tf.reshape(doc_embed_normalized, [-1, 3, self.doc_embedding_size]),
                3, 1)
            _loss = triplet_loss(self.anchor, self.positive, self.negative)
        return _loss

    def optimize(self, X):
        with tf.name_scope("optimize"):
            self.loss_op = self.loss(X)
            self.gradients = self.optimizer.compute_gradients(self.loss_op)
            apply_gradient_op = self.optimizer.apply_gradients(
                self.gradients, global_step=self.global_step)
        return apply_gradient_op

    def _convolv_on_embeddings(self, embeds, filter_sizes, nb_filter, kmax):
        """
        Create a convolution + k-max pool layer for each filter size, then concat and vectorize.
        embeds shape is [batch, (n_words or n_sents), embedding_size, 1]
        """
        pooled_outputs = []
        for fsize in filter_sizes:
            with tf.name_scope("conv-%s" % fsize):
                with tf.variable_scope(
                        "conv_weights_fsize-%s" % fsize, reuse=True):
                    weights_init = tf.get_variable('W')
                    bias_init = tf.get_variable('b')
                conv = tf.nn.conv2d(
                    embeds,
                    weights_init,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")

                h = tf.nn.relu(tf.nn.bias_add(conv, bias_init), name="relu")
#                 h shape is [batch, n_words - fsize + 1, 1, nb_filter]
#             tf.summary.histogram("relu", h)

            with tf.name_scope('%s-maxpool-fsize-%s' % (kmax, fsize)):
                # k-maxpooling over the outputs
                trans = tf.transpose(h, perm=[0, 2, 3, 1])
                values, indices = tf.nn.top_k(trans, k=kmax, sorted=False)
                pooled = tf.transpose(values, perm=[0, 3, 1, 2])
                # pooled shape is [batch, kmax, 1, nb_filter]
                pooled_outputs.append(pooled)

        with tf.name_scope('concat_and_vectorize'):
            # Combine all the pooled features
            h_pool = tf.concat(pooled_outputs, 3)
            # h_pool shape is [batch, kmax, 1, nb_filter*len(filter_sizes)]

            # Vectorize filters for each sent to get sent embeddings
            trans = tf.transpose(h_pool, perm=[0, 2, 3, 1])
            batch = tf.shape(embeds)[0]
            sent_embed = tf.reshape(trans, [batch, -1, 1])
            # sent_embed shape is [batch, kmax*nb_filter*len(filter_sizes), 1]

        return sent_embed

    def _create_sharable_weights(self, filter_sizes, embedding_size,
                                 nb_filter):
        """ Create sharable weights for each type of convolution """
        with tf.name_scope('sharable_weights'):
            for fsize in filter_sizes:
                with tf.variable_scope("conv_weights_fsize-%s" % fsize):
                    filter_shape = [fsize, embedding_size, 1, nb_filter]
                    initializer = tf.contrib.layers.xavier_initializer_conv2d(
                        uniform=True)
#                     initializer=tf.truncated_normal(stddev=0.1))
                    weights_init = tf.get_variable(
                        'W', filter_shape, initializer=initializer)
                    bias_init = tf.get_variable(
                        'b', initializer=tf.constant(0.1, shape=[nb_filter]))
    
    def init_summary(self):
        1
        
        
def triplet_loss(anchor_embed,
                 positive_embed,
                 negative_embed,
                 margin=0.2):
    """
    input: Three L2 normalized tensors of shape [None, dim], compute on a batch
    output: float
    """
    with tf.variable_scope('triplet_loss'):
        d_pos = tf.reduce_sum(tf.square(anchor_embed - positive_embed), 1)
        d_neg = tf.reduce_sum(tf.square(anchor_embed - negative_embed), 1)

        loss = tf.maximum(0., margin + d_pos - d_neg)
        loss = tf.reduce_mean(loss)

    return loss
    

In [272]:
triples_all = list(random_triples(sims, ids, num_epochs=1, seed=1))

100%|██████████| 368458/368458 [00:09<00:00, 40595.78it/s]


In [273]:
_triples, triples_test = train_test_split(triples_all, test_size=0.2, random_state=0)
triples_train, triples_val = train_test_split(_triples, test_size=0.3, random_state=0)

In [274]:
print(len(triples_train), len(triples_val), len(triples_test))

(428414, 183606, 153005)


In [289]:
start_time = time.time()

vocab_size, embedding_size = word_embeddings.shape
n_sents, n_words = 123, 40

with tf.Graph().as_default():
    tf.set_random_seed(0)

    session_conf = tf.ConfigProto(
        allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        anchor_batch, positive_batch, negative_batch, fnames = input_pipeline(
            triples_train[:64*4], batch_size=64, num_epochs=1)
        X = tf.reshape(
            tf.transpose([anchor_batch, positive_batch, negative_batch],
                         [1, 0, 2, 3]), [-1, n_sents, n_words],
            name='X')

        model = TextCNN(
            n_sents,
            n_words,
            vocab_size,
            embedding_size,
            sent_filter_sizes=[3, 4, 5],
            sent_nb_filter=50,
            doc_filter_sizes=[1, 2, 3],
            doc_nb_filter=10,
            sent_kmax=4,
            doc_kmax=10,
            learning_rate=0.001)
        train_op = model.optimize(X)

        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])

        saver = tf.train.Saver()

        # ===================summary====================
        # Create summaries to visualize weights
        tf.summary.scalar("loss", model.loss_op)
        for var in tf.trainable_variables():
            tf.summary.histogram(var.name.replace(':', '_'), var)
        # Summarize all gradients
        for grad, var in model.gradients:
            tf.summary.histogram(
                var.name.replace(':', '_') + '/gradient', grad)
        merged_summary_op = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(
            join(DATA_FOLDER, 'summary', 'train',
                 str(datetime.datetime.now())), sess.graph)
        # ===================summary====================

        # Assign word embeddings to variable W
        #!!!! index is shifted by 1
        sess.run(
            model.embedding_init,
            feed_dict={model.embedding_placeholder: word_embeddings})

        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                _, step, loss, summary, anchor = sess.run([
                    train_op, model.global_step, model.loss_op,
                    merged_summary_op, model.anchor
                ])
                current_step = tf.train.global_step(sess, model.global_step)
                train_writer.add_summary(summary, current_step)
                print(current_step, loss)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)

    saver.save(
        sess,
        join(DATA_FOLDER, 'models', '%s' % str(datetime.datetime.now())),
        global_step=current_step)

print("--- %s seconds ---" % (time.time() - start_time))

sent_embedding_size 600
doc_embedding_size 300
(1, 0.19535846)
(2, 0.12479226)
(3, 0.1467894)
(4, 0.16934134)
Done training -- epoch limit reached
--- 134.442246914 seconds ---


In [55]:
model_dir = join(DATA_FOLDER, 'models')
!ls {model_dir}

2017-08-25 11:36:11.921470-100.data-00000-of-00001
2017-08-25 11:36:11.921470-100.index
2017-08-25 11:36:11.921470-100.meta
checkpoint


# Testing

https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/4_Utils/save_restore_model.py

In [129]:
vocab_size, embedding_size = word_embeddings.shape
n_sents, n_words = 123, 40

doc_filter_sizes=[1, 2, 3]
doc_nb_filter=10
doc_kmax=10

with tf.Graph().as_default():
    tf.set_random_seed(0)

    session_conf = tf.ConfigProto(
        allow_soft_placement=True, log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        anchor_batch, positive_batch, negative_batch, fnames_batch = input_pipeline(
            triples_test[:64*20], batch_size=64, num_epochs=1, num_threads=1, shuffle=False)
        X = tf.reshape(
            tf.transpose([anchor_batch, positive_batch, negative_batch],
                         [1, 0, 2, 3]), [-1, n_sents, n_words],
            name='X')
        
        init_local = tf.local_variables_initializer()
        init_global = tf.global_variables_initializer()
        sess.run([init_global, init_local])
        
        # do not restore before global initialization, otherwise all weights are set to default !!!
        saver = tf.train.import_meta_graph(
            join(model_dir, '2017-08-25 11:36:11.921470-100.meta'), input_map={'X':X})
        saver.restore(sess, tf.train.latest_checkpoint(model_dir))
        graph = tf.get_default_graph()
    
        doc_embed_normalized = graph.get_operation_by_name('optimize/loss/doc_embed_normalized').outputs[0]
        
        anchor, positive, negative = tf.unstack(
            tf.reshape(doc_embed_normalized, [-1, 3, doc_kmax * doc_nb_filter * len(doc_filter_sizes)]),
            3, 1)
        _loss = triplet_loss(anchor, positive, negative)
                             
#         pprint([n.name for n in tf.get_default_graph().as_graph_def().node])

        doc_embeds, fnames = [], []

        # Start input enqueue threads.
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        try:
            while not coord.should_stop():
                [batch_embeds, loss, _names] = sess.run([doc_embed_normalized, _loss, fnames_batch])
                doc_embeds.append(batch_embeds)
                fnames += list(_names)
                print(batch_embeds.shape, loss)
        except tf.errors.OutOfRangeError:
            print('Done testing -- epoch limit reached')
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)

INFO:tensorflow:Restoring parameters from ../data/models/2017-08-25 11:36:11.921470-100


2017-08-25 14:15:39,698 : INFO : Restoring parameters from ../data/models/2017-08-25 11:36:11.921470-100


((192, 300, 1), 0.03564262)
((192, 300, 1), 0.015106287)
((192, 300, 1), 0.01525973)
((192, 300, 1), 0.027117617)
((192, 300, 1), 0.020871785)
((192, 300, 1), 0.020573003)
((192, 300, 1), 0.02336137)
((192, 300, 1), 0.023403049)
((192, 300, 1), 0.013923426)
((192, 300, 1), 0.030562393)
((192, 300, 1), 0.035025455)
((192, 300, 1), 0.023427295)
((192, 300, 1), 0.025457731)
((192, 300, 1), 0.0093168225)
((192, 300, 1), 0.020863641)
((192, 300, 1), 0.033610787)
((192, 300, 1), 0.018748417)
((192, 300, 1), 0.038897656)
((192, 300, 1), 0.02230488)
((192, 300, 1), 0.022732088)
Done testing -- epoch limit reached


In [130]:
fnames = np.concatenate(fnames)
doc_embeds = np.reshape(np.concatenate(doc_embeds), [-1, 300])

In [165]:
len(fnames)

3840

In [170]:
test_fn = fnames[6]
test_vec = doc_embeds[6]

In [171]:
dists = np.sum(np.square(doc_embeds - test_vec), 1)

In [175]:
ixs = dists.argsort()[:15]
fnames[ixs]

array(['../data/corpus/5984d324b6b1135cdf63851f.txt',
       '../data/corpus/5984d599b6b11375b46384f9.txt',
       '../data/corpus/5984d11ab6b113430363851e.txt',
       '../data/corpus/5984c946b6b11379a663852a.txt',
       '../data/corpus/5984d59fb6b11375c5638525.txt',
       '../data/corpus/5984d016b6b1133c0063852f.txt',
       '../data/corpus/5984c3b9b6b1133ef3638548.txt',
       '../data/corpus/5984d2f2b6b1135aa963854a.txt',
       '../data/corpus/5984dad2b6b1131d4b638520.txt',
       '../data/corpus/5984c93ab6b11379ac638509.txt',
       '../data/corpus/5984c968b6b1137bcd6384ee.txt',
       '../data/corpus/5984c54fb6b113508c6384fa.txt',
       '../data/corpus/5984ca5bb6b11304b8638501.txt',
       '../data/corpus/5984d32db6b1135cda638533.txt',
       '../data/corpus/5984d2e9b6b1135aaa6384fd.txt'], dtype=object)

In [155]:
ixs = dists.argsort()[-5:][::-1]
fnames[ixs]

array(['../data/corpus/5984c59cb6b11352d3638549.txt',
       '../data/corpus/5984c78bb6b11367c1638521.txt',
       '../data/corpus/5984c91fb6b1137864638533.txt',
       '../data/corpus/5984c93eb6b113798f638507.txt',
       '../data/corpus/5984c787b6b11367c6638505.txt'], dtype=object)

In [173]:
test_fn

'../data/corpus/5984d324b6b1135cdf63851f.txt'

In [213]:
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(0)
    sess = tf.Session()
    with sess.as_default():

        a = tf.convert_to_tensor([[[1,2],[3,4]],[[5,6],[7,8]],[[9,10],[11,12]]])
        b = tf.convert_to_tensor([[[13,14],[15,16]],[[17,18],[19,20]],[[21,22],[23,24]]])
        c = tf.convert_to_tensor([[[25,26],[27,28]],[[29,30],[31,32]],[[33,34],[35,36]]])
        
        d = tf.reshape(tf.transpose([a,b,c], [1,0,2,3]), [-1,2,2])
        
        sess.run(tf.global_variables_initializer())
        
        [res] = sess.run([d])
        print(res)

#         train_writer = tf.summary.FileWriter('../data/summary', sess.graph)

[[[ 1  2]
  [ 3  4]]

 [[13 14]
  [15 16]]

 [[25 26]
  [27 28]]

 [[ 5  6]
  [ 7  8]]

 [[17 18]
  [19 20]]

 [[29 30]
  [31 32]]

 [[ 9 10]
  [11 12]]

 [[21 22]
  [23 24]]

 [[33 34]
  [35 36]]]
